<a href="https://colab.research.google.com/github/naarkhoo/HafezRNN/blob/master/HafezGar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm divan_hafez.txt
!wget --show-progress --continue https://raw.githubusercontent.com/naarkhoo/HafezRNN/master/divan_hafez.txt

--2019-02-07 20:46:41--  https://raw.githubusercontent.com/naarkhoo/HafezRNN/master/divan_hafez.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 593410 (580K) [text/plain]
Saving to: ‘divan_hafez.txt’

divan_hafez.txt     100%[===================>] 579.50K  --.-KB/s    in 0.05s   

2019-02-07 20:46:41 (11.1 MB/s) - ‘divan_hafez.txt’ saved [593410/593410]



In [0]:
with open('divan_hafez.txt', 'rb') as f:
  txt = f.read().decode("utf-16")
  txt = txt.replace(u'\u200c','')

In [0]:
def remove_extra(txt):
  b = ''
  for item in txt.split('\n'):
    if item != '\r':
      b = item.replace('\r', '') + b
      #      a.append(item.replace('\r', ''))
  return(b)

In [0]:
import numpy as np 

# make everything number
output = np.asarray([ord(c) for c in txt], dtype=np.int32)
uvec = np.unique(output)

asciicode2unique_d = {ni: indi for indi, ni in enumerate(set(uvec))}
unique2asciicode_d = {y:x for x,y in asciicode2unique_d.items()}

In [22]:
[asciicode2unique_d[ord(c)] for c in txt[0:10]] 

[26, 18, 29, 4, 4, 4, 38, 2, 1, 2]

In [23]:
len(set(uvec))

48

In [36]:
import numpy as np
import six
import tensorflow as tf
import time
import os

# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

# SHAKESPEARE_TXT = '/content/shakespeare.txt'
SHAKESPEARE_TXT = 'divan_hafez.txt'

tf.logging.set_verbosity(tf.logging.INFO)

def transform(txt, pad_to=None):
  # drop any non-ascii characters
  #output = np.asarray([ord(c) for c in txt if ord(c) < 255], dtype=np.int32)
  txt = remove_extra(txt)
  #output = np.asarray([ord(c) for c in txt], dtype=np.int32)
  
  output = np.asarray([asciicode2unique_d[ord(c)] for c in txt], dtype=np.int32)

  if pad_to is not None:
    output = output[:pad_to]
    output = np.concatenate([
        np.zeros([pad_to - len(txt)], dtype=np.int32),
        output,
    ])
  return output

def training_generator(seq_len=100, batch_size=1024):
  """A generator yields (source, target) arrays for training."""
  #with tf.gfile.GFile(SHAKESPEARE_TXT, 'r') as f:
  #  txt = f.read()
    
  with open('divan_hafez.txt', 'rb') as f:
    txt = f.read().decode("utf-16")
    txt = txt.replace(u'\u200c','')
     
  tf.logging.info('Input text [%d] %s', len(txt), txt[:50])
    
  source = transform(txt)
  while True:
    offsets = np.random.randint(0, len(source) - seq_len, batch_size)

    # Our model uses sparse crossentropy loss, but Keras requires labels
    # to have the same rank as the input logits.  We add an empty final
    # dimension to account for this.
    yield (
        np.stack([source[idx:idx + seq_len] for idx in offsets]),
        np.expand_dims(
            np.stack([source[idx + 1:idx + seq_len + 1] for idx in offsets]),
            -1),
    )

six.next(training_generator(seq_len=100, batch_size=5))

INFO:tensorflow:Input text [295224] غزل   ۱

الا يا ايها الساقی ادر کاسا و ناولها
ک


(array([[33,  4,  8, 19,  4, 13, 28,  4, 14, 15, 30, 10,  4,  7, 19, 10,
          6, 15, 30,  4,  8, 32, 20, 10,  4, 17, 33, 22, 32,  4, 15,  7,
         17,  7, 29, 19, 29,  7, 30,  4, 17,  7, 15, 17,  4, 25, 34, 20,
          4, 31, 28, 15,  4,  9, 33, 20,  4,  9, 32,  4,  0, 33, 31,  4,
          6,  8, 14, 33, 17,  4, 31, 30,  7, 31, 15, 47, 34, 17,  7, 31,
         32,  4, 19, 17,  4, 30,  9, 31,  4, 32, 31, 17, 35,  4, 31, 31,
          5,  4, 33,  4],
        [25, 29,  4,  7, 20,  9,  4, 15,  7, 17, 30,  4,  5, 31, 12, 32,
          7, 10,  7,  4, 18,  4,  7, 20,  9,  4, 33,  4,  0, 32, 17, 32,
          4, 17,  7, 32, 10,  4, 47, 17, 18, 17,  4, 33,  4,  5, 33, 32,
         17,  4,  9, 31, 30,  8,  7, 18,  9, 20,  4, 34,  9,  4, 15, 30,
          4, 25, 31,  7, 31,  4,  7, 35,  4, 10, 17,  9,  4, 20, 32, 17,
          6, 20, 33,  8,  4, 30, 31, 15,  7, 33, 17, 35,  4, 15,  7, 17,
         30,  4,  8, 19],
        [17, 15, 30,  4,  0, 20, 30, 30,  4,  9, 32,  4,  8,  7,  4, 19,

In [0]:
EMBEDDING_DIM = 512

def lstm_model(seq_len=100, batch_size=None, stateful=True):
  """Language model: predict the next word given the current word."""
  source = tf.keras.Input(
      name='seed', shape=(seq_len,), batch_size=batch_size, dtype=tf.int32)

  #embedding = tf.keras.layers.Embedding(input_dim=256, output_dim=EMBEDDING_DIM)(source)
  embedding = tf.keras.layers.Embedding(input_dim=50, output_dim=EMBEDDING_DIM)(source)
  
  lstm_1 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(embedding)
  lstm_2 = tf.keras.layers.LSTM(EMBEDDING_DIM, stateful=stateful, return_sequences=True)(lstm_1)
#  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='softmax'))(lstm_2)
  predicted_char = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(50, activation='softmax'))(lstm_2)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_char])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
  return model

In [38]:
tf.keras.backend.clear_session()

training_model = lstm_model(seq_len=100, batch_size=128, stateful=False)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.fit_generator(
    training_generator(seq_len=100, batch_size=1024),
    steps_per_epoch=100,
    epochs=10,
)
tpu_model.save_weights('/tmp/bard_hafez.h5', overwrite=True)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.69.29.114:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4595241092013431743)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 15567322301889423376)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 16675418112568878402)
INFO:tensorflow:*** Avai

In [47]:
BATCH_SIZE = 5
PREDICT_LEN = 45

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard_hafez.h5')

# We seed the model with our initial string, copied BATCH_SIZE times

seed_txt = 'الا یا ایه الساقی'

print(type(seed_txt))
seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)
print(seed)

<class 'str'>
[[ 7 29  7  4 35  7  4  7 35 32  4  7 29 19  7 28 35]
 [ 7 29  7  4 35  7  4  7 35 32  4  7 29 19  7 28 35]
 [ 7 29  7  4 35  7  4  7 35 32  4  7 29 19  7 28 35]
 [ 7 29  7  4 35  7  4  7 35 32  4  7 29 19  7 28 35]
 [ 7 29  7  4 35  7  4  7 35 32  4  7 29 19  7 28 35]]


In [0]:
for i in range(len(seed_txt)-1):
  prediction_model.predict(seed[:, i:i + 1])

In [0]:
# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])

# Now we can accumulate predictions!
predictions = [seed[:, -1:]]
for i in range(PREDICT_LEN):
  last_word = predictions[-1]
  next_probits = prediction_model.predict(last_word)[:, 0, :]
  
  # sample from our output distribution
  next_idx = [
      np.random.choice(50, p=next_probits[i])
      for i in range(BATCH_SIZE)
  ]
  predictions.append(np.asarray(next_idx, dtype=np.int32))
 

In [50]:
for i in range(BATCH_SIZE):
  print('PREDICTION %d\n\n' % i)
  p = [predictions[j][i] for j in range(PREDICT_LEN)]
  #print(p)
  print(''.join([str(chr(unique2asciicode_d[item])) for item in p[1::]]))
  generated = ''.join([chr(c) for c in p])
  print(generated)
  assert len(generated) == PREDICT_LEN, 'Generated text too short'

PREDICTION 0


 می با فلک و کشتی هر آن کس که کمان ابروی ميخ
##	!	
# 		 	!#"
PREDICTION 1


 مدارجهان عيب خود خوشش باش که خيری به جای خو
# "!!	 "# #!
PREDICTION 2


 مدار همعزم دراز که بردارد زمانی برقع از روي
# 	 #!"
PREDICTION 3


 مدار کجاستآن کس است اهل رخشان به جای من سرو
#	
	
  #!
PREDICTION 4


 که مهمت است آب روان آيد و بلاکش باشدغزل    
#	  

!"!	


'کند\r\n\r\nغبار دارد که فلک '

In [0]:
BATCH_SIZE = 5
PREDICT_LEN = 250

# Keras requires the batch size be specified ahead of time for stateful models.
# We use a sequence length of 1, as we will be feeding in one character at a 
# time and predicting the next character.
prediction_model = lstm_model(seq_len=1, batch_size=BATCH_SIZE, stateful=True)
prediction_model.load_weights('/tmp/bard.h5')

# We seed the model with our initial string, copied BATCH_SIZE times


OSError: ignored

In [0]:
seed_txt = 'Looks it not like the king?  Verily, we must go! '
#seed_txt = 'الساقی ادر کاسا و ناولها'

seed = transform(seed_txt)
seed = np.repeat(np.expand_dims(seed, 0), BATCH_SIZE, axis=0)
seed

array([[ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  32,  86, 101, 114, 105, 108, 121,  44,  32, 119, 101,
         32, 109, 117, 115, 116,  32, 103, 111,  33,  32],
       [ 76, 111, 111, 107, 115,  32, 105, 116,  32, 110, 111, 116,  32,
        108, 105, 107, 101,  32, 116, 104, 101,  32, 107, 105, 110, 103,
         63,  32,  3

In [0]:
# First, run the seed forward to prime the state of the model.
prediction_model.reset_states()
for i in range(len(seed_txt) - 1):
  prediction_model.predict(seed[:, i:i + 1])


In [0]:
with open('divan_hafez.txt', 'rb') as f:
  contents = f.read()
  contents = contents.decode("utf-16")

bb = contents.replace(u'\u200c','')
bb[1:50]

'زل   ۱\r\n\r\nالا يا ايها الساقی ادر کاسا و ناولها\r\nک'